In [1]:
import requests
import pandas as pd
import io
import re
import html
import PyPDF2

In [2]:
url = "https://docs.google.com/spreadsheets/d/1ur7M6kkW_gNyziO-KEwVtiePvW_Zf-rv6gXiZtOKIVE/export?output=xlsx"

In [3]:
response = requests.get(url)

In [4]:
buf = io.BytesIO(response.content)

In [5]:
excel = pd.read_excel(buf, usecols=['Resultado', 'Link']).dropna()

In [94]:
excel

,Resultado,Link
0,Procedente,https://drive.google.com/file/d/1PHACn-CxSkHCz...
1,Procedente,https://drive.google.com/file/d/15EV2gp0pRyzRJ...
2,Improcedente,https://drive.google.com/file/d/1AYXoUbtRnPRAP...
3,Procedente,https://drive.google.com/file/d/1lqIrqhGjB7DWX...
4,Procedente,https://drive.google.com/file/d/1JdTLSF8OQIpT8...
5,Improcedente,https://drive.google.com/file/d/1pQrbT6AVsKEOq...
6,Improcedente,https://drive.google.com/file/d/1AmMR4M2Y_MmOP...
7,Improcedente,https://drive.google.com/file/d/1KsnMrUZOhfsbb...
8,Procedente,https://drive.google.com/file/d/1vSAzQ3cWmhBL2...
9,Procedente,https://drive.google.com/file/d/1OyVU6gJ1S91Z3...


In [7]:
def binnaryPDFtoText(binnary):
    buf = io.BytesIO(binnary)
    
    read_pdf = PyPDF2.PdfFileReader(buf)
    text = ""
    for i in range(read_pdf.getNumPages()):
        page = read_pdf.getPage(i)
        page_content = page.extractText()
        text += ' '.join(page_content)
    return text

In [28]:
def download_from_google_drive(file_url):
    key = file_url[32:-17]
    
    hub_url = "https://docs.google.com/uc?export=download"
    session = requests.Session()
    response = session.get(hub_url, params = { 'id' : key }, stream = True)
    
    return binnaryPDFtoText(response.content)

In [13]:
def download_from_tjmg_tipo1(link):
    numero, ano, codigoOrigem = re.search('0*(\d+)\d-\d+.\d{2}(\d{2}).\d+.\d+.0*(\d+)', link).group(1,2,3)
    url = f"https://www5.tjmg.jus.br/jurisprudencia/relatorioEspelhoAcordao.do?inteiroTeor=true&ano={ano}&ttriCodigo=1&codigoOrigem={codigoOrigem}&numero={numero}&sequencial=1&sequencialAcordao=0"

    response = requests.get(url)
    return binnaryPDFtoText(response.content)

In [10]:
def download_from_tjmg_tipo2(link):
    response = requests.get(link)
    texto = response.text
    
    if(texto.startswith('%PDF')):
        return None
    
    texto = re.sub('<[^>]+>', "",texto)
    texto = texto.replace('\n', ' ')
    return html.unescape(texto) 

In [31]:
def download(link):
    text = ""
    try:
        if('drive' in link):
            text = download_from_google_drive(link)    
        elif('tjmg' in link):
            if(re.search('numeroUnico=[\d.-]+',link)):
                text = download_from_tjmg_tipo1(link)
            else:
                text = None
        else:
            text = None
    except:
        text=None
    return text

In [32]:
temp = excel.Link.apply(download)

In [105]:
temporario = pd.concat([excel.Link, temp],axis=1)

In [157]:
temporario

,link,text
0,https://drive.google.com/file/d/1PHACn-CxSkHCz...,T r i b u n a l d e J u s t i ç a d e ...
1,https://drive.google.com/file/d/15EV2gp0pRyzRJ...,T r i b u n a l d e J u s t i ç a d e ...
2,https://drive.google.com/file/d/1AYXoUbtRnPRAP...,T r i b u n a l d e J u s t i ç a d e ...
3,https://drive.google.com/file/d/1lqIrqhGjB7DWX...,T r i b u n a l d e J u s t i ç a d e ...
4,https://drive.google.com/file/d/1JdTLSF8OQIpT8...,T r i b u n a l d e J u s t i ç a d e ...
5,https://drive.google.com/file/d/1pQrbT6AVsKEOq...,T r i b u n a l d e J u s t i ç a d e ...
6,https://drive.google.com/file/d/1AmMR4M2Y_MmOP...,T r i b u n a l d e J u s t i ç a d e ...
7,https://drive.google.com/file/d/1KsnMrUZOhfsbb...,T r i b u n a l d e J u s t i ç a d e ...
8,https://drive.google.com/file/d/1vSAzQ3cWmhBL2...,T r i b u n a l d e J u s t i ç a d e ...
9,https://drive.google.com/file/d/1OyVU6gJ1S91Z3...,T r i b u n a l d e J u s t i ç a d e ...


In [149]:
temporario.text.groupby(lambda t: not not t).count()

False     1
True     24
Name: text, dtype: int64

In [108]:
temporario.columns = ['link', 'text']

In [154]:
deu_errado = temporario[temporario.text.apply(lambda t: not t)].link

In [159]:
deu_errado.index

Int64Index([25, 26, 27, 28, 29, 42, 44, 47, 48, 49, 50, 52, 53, 54, 55, 56,
            57],
           dtype='int64')

In [155]:
deu_errado.shape

(17,)

In [52]:
temp1 = temp.str.replace("   ","@@@").str.replace(" ","").str.replace("@@@"," ").dropna()

In [143]:
temp1

0     Tribunal de Justiça de Minas Gerais1.0000.21.1...
1     Tribunal de Justiça de Minas Gerais1.0106.19.0...
2     Tribunal de Justiça de Minas Gerais1.0000.21.1...
3     Tribunal de Justiça de Minas Gerais1.0000.21.0...
4     Tribunal de Justiça de Minas Gerais1.0702.15.0...
5     Tribunal de Justiça de Minas Gerais1.0000.21.0...
6     Tribunal de Justiça de Minas Gerais1.0086.18.0...
7     Tribunal de Justiça de Minas Gerais1.0000.21.0...
8     Tribunal de Justiça de Minas Gerais1.0000.19.1...
9     Tribunal de Justiça de Minas Gerais1.0000.20.5...
10    Tribunal de Justiça de Minas Gerais1.0000.19.1...
11    Tribunal de Justiça de Minas Gerais1.0000.20.5...
12    Tribunal de Justiça de Minas Gerais1.0000.20.5...
13    Tribunal de Justiça de Minas Gerais1.0000.21.0...
18    Tribunal de Justiça de Minas Gerais1.0702.15.0...
19    Tribunal de Justiça de Minas Gerais1.0000.19.1...
20    Tribunal de Justiça de Minas Gerais1.0000.20.0...
21    Tribunal de Justiça de Minas Gerais1.0000.

In [93]:
temp1.shape

(25,)

In [68]:
df = pd.concat([excel.Resultado, temp1],axis=1,join='inner')

In [69]:
df.columns = ['Resultado', 'text']
df

,Resultado,text
0,Procedente,Tribunal de Justiça de Minas Gerais1.0000.21.1...
1,Procedente,Tribunal de Justiça de Minas Gerais1.0106.19.0...
2,Improcedente,Tribunal de Justiça de Minas Gerais1.0000.21.1...
3,Procedente,Tribunal de Justiça de Minas Gerais1.0000.21.0...
4,Procedente,Tribunal de Justiça de Minas Gerais1.0702.15.0...
5,Improcedente,Tribunal de Justiça de Minas Gerais1.0000.21.0...
6,Improcedente,Tribunal de Justiça de Minas Gerais1.0086.18.0...
7,Improcedente,Tribunal de Justiça de Minas Gerais1.0000.21.0...
8,Procedente,Tribunal de Justiça de Minas Gerais1.0000.19.1...
9,Procedente,Tribunal de Justiça de Minas Gerais1.0000.20.5...


In [70]:
df.text = df.text.str.replace("Tribunal de Justiça de Minas Gerais","").str.replace(".*Data da Publicação:(EMENTA: *)?", "", regex=True)

In [71]:
df

,Resultado,text
0,Procedente,APELAÇÃO CÍVEL. AÇÃO DECLARATÓRIA DE INEXISTÊN...
1,Procedente,APELAÇÃO CÍVEL. CONEXÃO. FEITOS JULGADOS CONJU...
2,Improcedente,APELAÇÃO CÍVEL - AÇÃO DECLARATÓRIA DE INEXISTÊ...
3,Procedente,APELAÇÃO CÍVEL. CONSUMIDOR. INSCRIÇÃO EMCADAST...
4,Procedente,APELAÇÃO CÍVEL. AÇÃO DECLARATÓRIA DE INEXISTÊN...
5,Improcedente,APELAÇÃO CÍVEL - AÇÃO DECLARATÓRIA DE INEXISTÊ...
6,Improcedente,APELAÇÃO CÍVEL. AÇÃO INDENIZATÓRIA. SERVIÇO DE...
7,Improcedente,APELAÇÃO CÍVEL - AÇÃO DECLARATÓRIA DE INEXISTÊ...
8,Procedente,APELAÇÃO CÍVEL - AÇÃO DECLARATÓRIA DE INEXISTÊ...
9,Procedente,APELAÇÃO CÍVEL - AÇÃO DECLARATÓRIA DEINEXIGIBI...


In [86]:
df.Resultado = (df.Resultado == "Improcedente").values.astype('i')

In [89]:
df.to_csv("./dataframe.csv", index=False)